# Fashion MNIST复习

最新的TensorFlow已经使用fashion mnist数据集替代mnist数据集作为起始教程了。

我最近学了斯坦福的理论基础，现在准备回过头来复习一遍基础，以此加深理解。

以前的mnist数据集是手写数字的图片，现在的fashion mnist数据集是一些衣物的图片，但是两者图片的大小都是一致的，均为`28*28`。


## 加载数据集

现在，先加载数据集：

In [ ]:
import keras


fashion_mnist = keras.datasets.fashion_mnist
# 第一次会从网络下载数据集，目录在当前用户目录，以我为例，在/home/allen/.keras/datasets/fashion_mnist
train_images, train_labels, test_images, test_labels = fashion_mnist.load_data()
# train_images的形状为[60000, 28, 28]，代表60000张图片，每张图片为28*28像素
# train_labels是一个长度为60000的列表，每个值代表对用train image的类别
# test_images的形状是[10000, 28, 28]，test_labels是长度为10000的列表

# 类别的字符描述，把train labels提供的数字类别作为索引，可以取出对应的类别字符描述
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 看看图片是什么样的


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # 输出二值化图像
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])

![fashion_mnist_train_image_0](images/fashion_mnist_train_image_0.png)
![fashion_mnist_train_images_25](images/fashion_mnist_train_images_25.png)

## 构建模型

我们使用keras的sequential api构建模型:


In [21]:
import keras
import tensorflow as tf

# 使用keras的序贯模型
model = keras.Sequential()
# 将`28*28`的二维图像，展平成一维
flatten = keras.layers.Flatten(input_shape=(28, 28))
model.add(flatten)

# 添加全连接层
dense = keras.layers.Dense(128, activation=keras.activations.relu)
model.add(dense)

# 添加输出层，使用softmax函数进行多分类，获得每个类别的概率
output = keras.layers.Dense(10, activation=keras.activations.softmax)
model.add(output)

# 编译模型
# 1.使用adam作为优化器
# 2.使用loss=keras.losses.sparse_categorical_crossentropy作为损失，
#  如果使用tensorflow作为后端，它实际上就是tf.nn.sparse_softmax_cross_entropy_with_logits，
#  因为我们的输出是整数，所以使用这个loss
# 3.metrics我们使用accuracy，用来统计正确率
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

print("Model build.")
print("Output shape of flatten layer is:")
print(flatten.output_shape)

print("Output shape of dense layer is:")
print(dense.output_shape)

print("Output shape of output layer is:")
print(output.output_shape)

Model build.
Output shape of flatten layer is:
(None, 784)
Output shape of dense layer is:
(None, 128)
Output shape of output layer is:
(None, 10)


关于上面的loss，还需要多说一句。
tf.nn.sparse_softmax_cross_entropy_with_logits的参数需要的是logits，而keras.losses.sparse_categorical_crossentropy的参数需要的是probabilities。keras.losses.sparse_categorical_crossentropy有个参数from_logits，如果是False(默认值)，就需要对第一个输入参数进行一次对数运算，然后reshape，转化成logits。

```python

output = tf.log(output)
output_shape = output.get_shape()
logits　＝　tf.reshape(output, [-1, int(output_shape[-1])])
```

## 训练模型

使用keras训练模型很简单：


In [ ]:
# 使用train_images和对应的标签train_labels训练5个轮回
model.fit(train_images, train_labels, epochs=5)

## 估算模型的准确率

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test loss:',test_loss)
print('Test accuracy:', test_acc)

## 预测

我们使用test_images作为输入，预测分类：


In [ ]:
import numpy as np


predictions = model.predict(test_images)
predict_num_classes = []
for i in range(len(test_labels)):
    # 使用argmax操作，从softmax的输出概率中，选择最有可能的类别
    prediction = np.argmax(predictions[i])
    print(prediction)
    predict_num_classes.append(prediction)

print(predict_num_classes)